In [1]:
import pandas as pd
from pandas import DataFrame, read_csv
import networkx as nx

In [ ]:
with open('atari_edges_fixed.csv', 'a', encoding="utf8") as file_1, open('atari_edges.csv', "r", encoding="utf8") as file_2:
    for line in file_2:
        if ',quotes' not in line:
            file_1.write(line)

In [9]:
df = pd.read_csv('atari_edges_fixed.csv', header = None)
df.columns = ['src', 'dst', 'tema', 'type']
df.head()

,src,dst,tema,type
0,supercat,Trench,Market,replies_to_author
1,Clint Thompson,Trench,Market,replies_to_author
2,holygrailvideogames.com,Trench,Market,replies_to_author
3,Mister-VCS,alex_79,Console Systems,replies_to_author
4,PressureCooker2600,Red 5,HW,replies_to_author


In [15]:
df_simple = df[['src', 'tema']]

In [17]:
df = df_simple.groupby(df_simple.columns.tolist()).size().reset_index().rename(columns={0:'weight'})


In [18]:
df.head()

,src,tema,weight
0,(BK),HW,1
1,(J),Social,3
2,* Alternative Reality *,Games,2
3,* Alternative Reality *,HW,3
4,* Alternative Reality *,Social,1


In [19]:
df

,src,tema,weight
0,(BK),HW,1
1,(J),Social,3
2,* Alternative Reality *,Games,2
3,* Alternative Reality *,HW,3
4,* Alternative Reality *,Social,1
5,***astro_boy***,Console Systems,9
6,***astro_boy***,Dev,2
7,***astro_boy***,Games,2
8,***astro_boy***,HW,1
9,***astro_boy***,Market,13


In [20]:
G = nx.Graph()
for index, row in df.iterrows():
    
    G.add_node(row['src'], bipartite=0)
    G.add_node(row['tema'], bipartite=1)
    
    if G.has_edge(row['src'], row['tema']):
                        G[row['src']][row['tema']]['weight'] += row['weight']
    else:
        G.add_edge(row['src'], row['tema'], weight = row['weight'])

In [21]:
nx.is_bipartite(G)

True

In [22]:
len(list(G.edges()))

51855

In [23]:
nx.write_graphml_lxml(G, "user_topics_AA.graphml")

In [24]:
top_nodes = {n for n, d in G.nodes(data=True) if d['bipartite']==1}
bottom_nodes = set(G) - top_nodes

In [25]:
def forum_projection(B, nodes):
    if B.is_directed():
        pred = B.pred
        G = nx.DiGraph()
    else:
        pred = B.adj
        G = nx.Graph()
    G.graph.update(B.graph)
    G.add_nodes_from((n, B.nodes[n]) for n in nodes)
    for u in nodes:
        unbrs = set(B[u]) #nodos a los que conecta del otro nivel
        nbrs2 = set(n for nbr in unbrs for n in B[nbr] if n != u) #nodos con los que coincide
        for v in nbrs2:
            vnbrs = set(pred[v])
            common_degree = ((len(B[n]), B[u][n]['weight'] + B[v][n]['weight']) for n in unbrs & vnbrs)
            weight = sum(wt / (deg - 1) for deg, wt in common_degree if deg > 1)
            G.add_edge(u, v, weight=weight)
    return G

In [ ]:
G_top = forum_projection(G, top_nodes)
G_bottom = forum_projection(G, bottom_nodes)

In [ ]:
import community # al igual hace falta instalar pip install python-louvain
partition = community.best_partition(G_bottom)

In [ ]:
df['community'] = df.apply(lambda row: partition[row.Source], axis=1)

In [ ]:
df.to_csv('communities_AA_tab.csv', sep ='\t')

In [ ]:
df.groupby(['community']).agg({'weight': 'sum'}).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
pesos = df.groupby(['community', 'topic_id']).agg({'weight': 'sum'})
pesos['percentage'] = pesos['weight']/pesos.groupby('community')['weight'].transform('sum')

In [ ]:
pesos